# Baseline Model based on U-Net

In [1]:
import os
os.chdir("..")

## Training

In [2]:
from baseline.trainer import Trainer

trainer = Trainer()

Segmentation Models: using `tf.keras` framework.


In [ ]:
%time history = trainer.train_from_simple_dataloader()

In [ ]:
plt.figure(figsize=(30, 5))
plt.subplot(121)
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

## Prediction

In [3]:
import segmentation_models as sm
from constants import TEST_DIR
from baseline.dataloader import SimpleDataLoader
from baseline.data_augmentation import DataAugmentation
from baseline.helper import Visualisation

simple_data_loader = SimpleDataLoader(
    images_path=os.path.join(TEST_DIR, "images"),
    preprocessing=DataAugmentation().get_preprocessing(sm.get_preprocessing(Trainer.BACKBONE))
)

for test_image in simple_data_loader.get_images()[:3]:
    print(test_image)
    predicted_mask = trainer.get_model().predict(test_image).round()
    print(predicted_mask)
    
    Visualisation().show(
        image=image.squeeze(),
        predicted_mask=predicted_mask[..., 0].squeeze()
    )

[[[ 31  27  28]
  [ 32  28  29]
  [ 30  26  27]
  ...
  [173 129 146]
  [174 130 147]
  [172 129 146]]

 [[ 31  27  28]
  [ 32  28  29]
  [ 30  26  27]
  ...
  [171 127 144]
  [171 127 144]
  [170 127 144]]

 [[ 30  26  27]
  [ 32  28  29]
  [ 29  27  30]
  ...
  [169 125 142]
  [171 127 144]
  [172 129 146]]

 ...

 [[180 165 170]
  [181 166 171]
  [184 169 174]
  ...
  [183 164 170]
  [182 165 171]
  [182 165 171]]

 [[178 161 167]
  [178 161 167]
  [183 168 173]
  ...
  [184 165 171]
  [183 164 170]
  [183 164 170]]

 [[179 162 168]
  [179 162 168]
  [184 169 174]
  ...
  [184 165 171]
  [184 165 171]
  [184 165 171]]]


ValueError: in user code:

    /usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1461 run_step  **
        outputs = mo